# [Module 6.2] inference on model that is invoked ny Kubeflow

이 노트북은 Kubeflow Pipeline에서 SageMaker에서 Training을 요청하셔 생성한 모델 아티펙트를 가져와서 추론을 합니다.



---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [1]:
!pip install -q tensorflow==2.1.0

In [2]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## SageMaker Train Model Invoked by KubeFlow

엔드포인트를 만들기 위해 아래 두가지를 준비 합니다.
- SageMaker Training Job을 가져와서 Model Artifact를 사용 합니다.
- "4.2.1.Make-Custom-Inference-Image-ECR.ipynb" 에서 생성하여 ECR에 등록한 추론 이미지를 가져옵니다.

In [3]:
model_data = 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz'
print('sagemaker train job from KubeFlow: \n', model_data)

sagemaker train job from KubeFlow: 
 s3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz


In [4]:
inference_image = '343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## 엔드포인트 생성

In [5]:
%%time

from sagemaker.tensorflow.serving import Model

model = Model(model_data= model_data,
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


---------------!CPU times: user 1min 3s, sys: 10.3 s, total: 1min 14s
Wall time: 8min 45s


## Predictor Creation on the Endpoint

In [6]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-19-03-07-35-911


In [7]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [8]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [9]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
930,good morning i'm feeling good this today are you,4
5989,my mood is all over the place today,8
7244,been single for 2 years,8
7514,mood,1
5706,the emojis are ready gswmediaday,2
4279,i love them to,1
8141,lol bro i can't believe i just embarrassed mys...,1
1298,this is what's happening while you were study...,3
6087,i hate how my air cuts on and off son,8
6406,stubbs mill nearing completion looking amazin...,5


In [10]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 5 이모티콘 추천

In [12]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    tweets = [tweet]    
    predicted_classes = predictor.predict(tweets)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('-------------------------------------------')        
    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        ))    

-------------------------------------------
tweet: good morning i'm feeling good this today are you  
Ground_truth- 4:😊
 
Prediction: 4,😊,1,💕,0,❤,5,😍,2,🔥 
 
-------------------------------------------
tweet:  my mood is all over the place today 
Ground_truth- 8:🙄
 
Prediction: 5,😍,4,😊,3,😂,7,😭,6,😩 
 
-------------------------------------------
tweet:  been single for 2 years 
Ground_truth- 8:🙄
 
Prediction: 6,😩,4,😊,3,😂,8,🙄,7,😭 
 
-------------------------------------------
tweet:  mood  
Ground_truth- 1:💕
 
Prediction: 4,😊,6,😩,5,😍,7,😭,8,🙄 
 
-------------------------------------------
tweet:  the emojis are ready gswmediaday 
Ground_truth- 2:🔥
 
Prediction: 5,😍,0,❤,4,😊,2,🔥,8,🙄 
 
-------------------------------------------
tweet: i love them to  
Ground_truth- 1:💕
 
Prediction: 0,❤,5,😍,1,💕,2,🔥,4,😊 
 
-------------------------------------------
tweet: lol bro i can't believe i just embarrassed myself but anything for the woman that inspires me in my everyday life 
Ground_truth- 1:💕
 
Pre